# Rumor Propagation Model
Simulating Various Scenarios of Rumor Propagation

CSCI 115 O Group 4

## Model

In [1]:
pip install agentpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.0/758.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00


In [2]:
# Model design
import agentpy as ap
import numpy as np
import random as rand

# Visualization
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import IPython

matplotlib.rcParams['animation.embed_limit'] = 2**128

In [3]:
# Class

class RumorModel(ap.Model):
  def setup(self):
    # Agents
    nPersons = int(self.p['Population Density'] * (self.p.size**2))
    persons = self.agents = ap.AgentList(self, nPersons)
    # Grid
    self.city = ap.Grid(self, [self.p.size]*2, track_empty=True)
    self.city.add_agents(persons, random=True, empty=True)
    # Time Steps
    self.timeStep = 0
    # Agents' Initial Condition
    self.agents.condition = 0

  def update(self):
    # I: Ignorant, RS: Rumor Spreader, TS: Truth Spreader, RR: RS Removed, TR: TS Removed
    pop = 0
    for i in range(0, 7):
      pop += len(self.agents.select(self.agents.condition == i))
    # 0: I, 1: NEW RS, 2: RS, 3: NEW TS, 4: TS, 5: RR, 6: TR
    self['I'] = len(self.agents.select(self.agents.condition == 0))/pop
    self['RS'] = (len(self.agents.select(self.agents.condition == 1)) + len(self.agents.select(self.agents.condition == 2)))/pop
    self['TS'] = (len(self.agents.select(self.agents.condition == 3)) + len(self.agents.select(self.agents.condition == 4)))/pop
    self['RR'] = len(self.agents.select(self.agents.condition == 5))/pop
    self['TR'] = len(self.agents.select(self.agents.condition == 6))/pop
    self.record('I')
    self.record('RS')
    self.record('TS')
    self.record('RR')
    self.record('TR')
  
  def step(self):
    ignorant = self.agents.select(self.agents.condition == 0)
    new_RS = self.agents.select(self.agents.condition == 1)
    rumor_Spreader = self.agents.select(self.agents.condition == 1)
    rumor_Spreader += self.agents.select(self.agents.condition == 2)
    new_TS = self.agents.select(self.agents.condition == 3)
    truth_Spreader = self.agents.select(self.agents.condition == 3)
    truth_Spreader += self.agents.select(self.agents.condition == 4)
    removed_RS = self.agents.select(self.agents.condition == 5)
    removed_TS = self.agents.select(self.agents.condition == 6) 

    # Converting NEW RS to RS
    for nr in new_RS:
      nr.condition = 2
    
    # Converting NEW TS to TS
    for nt in new_TS:
      nt.condition = 4
    
    # Change of State of I
    for i in ignorant:
      rng = rand.uniform(0, 1)
      n = self.city.neighbors(i).to_list()
      rs_count = n.select(n.condition == 2)
      ts_count = n.select(n.condition == 4)
      if (len(rs_count) + len(ts_count)) > 0:
        if (len(rs_count)) > len(ts_count):
          if rng < self.p['Probability of Accepting the Rumor']:
            i.condition = 1
        elif (len(rs_count)) == len(ts_count):
          pass
        elif (len(rs_count)) < len(ts_count):
          if rng < self.p['Probability of Accepting the Truth']:
            i.condition = 3

    
    # Change of State of RS
    if self.timeStep == self.p['Rumor Spread Start']:
      # At the said time step, the initial rumor spreaders are introduced. No spread occurs yet.
      ignorant.random(self.p['Initial Rumor Spreaders']).condition = 2
    elif self.timeStep > self.p['Rumor Spread Start']:
      # At the following time steps, the rumor is spread.
      for rs in rumor_Spreader:
        rng = rand.uniform(0, 1)
        n = self.city.neighbors(rs).to_list()
        rs_count = n.select(n.condition == 2)
        ts_count = n.select(n.condition == 4)
        if (len(rs_count) + len(ts_count)) > 0:
          if (len(rs_count)) > len(ts_count):
            pass
          elif (len(rs_count)) == len(ts_count):
            pass
          elif (len(rs_count)) < len(ts_count):
            if rng < self.p['Probability of Switching Sides after Contact with a Truth Spreader']:
              rs.condition = 3

    # Change of State of TS
    if self.timeStep == self.p['Truth Spread Start']:
      # At the said time step, the initial truth spreaders are introduced. No spread occurs yet.
      ignorant.random(self.p['Initial Truth Spreaders']).condition = 4
    elif self.timeStep > self.p['Truth Spread Start']:
      # At the following time steps, the truth is spread.
      for ts in truth_Spreader:
        rng = rand.uniform(0, 1)
        n = self.city.neighbors(ts).to_list()
        rs_count = n.select(n.condition == 2)
        ts_count = n.select(n.condition == 4)
        if (len(rs_count) + len(ts_count)) > 0:
          if (len(rs_count)) > len(ts_count):
            if rng < self.p['Probability of Switching Sides after Contact with a Rumor Spreader']:
              ts.condition = 1
          elif (len(rs_count)) == len(ts_count):
            pass
          elif (len(rs_count)) < len(ts_count):
            pass

    # RS losing interest
    for rs in rumor_Spreader:
      if rand.uniform(0, 1) < self.p['Probability of Losing Interest for Rumor Spreaders']:
        rs.condition = 5

    # TS losing interest
    for ts in truth_Spreader:
      if rand.uniform(0, 1) < self.p['Probability of Losing Interest for Truth Spreaders']:
        ts.condition = 6

    # RR forgetting the rumor
    for rs in removed_RS:
      if rand.uniform(0, 1) < self.p['Probability of Forgetting the Rumor']:
        rs.condition = 0
    
    # TR forgetting the rumor
    for ts in removed_TS:
      if rand.uniform(0, 1) < self.p['Probability of Forgetting the Truth']:
        ts.condition = 0
    
    # Stopping the simulation when there are no more Rumor Spreaders
    if self.timeStep > self.p['Truth Spread Start']:
      if (len(rumor_Spreader) == 0):
        self.stop()
      elif (len(truth_Spreader) == 0):
        self.stop()
    
    # Adding 1 to self.timeStep
    self.timeStep += 1

  def end(self):
    # Rumor Believers: Rumor Spreaders & RS Removed
    rumor_Believers = len(self.agents.select(self.agents.condition == 1)) + len(self.agents.select(self.agents.condition == 3))
    # Truth Believers: Truth Spreaders & TS Removed
    truth_Believers = len(self.agents.select(self.agents.condition == 2)) + len(self.agents.select(self.agents.condition == 4))
    self.report('Percentage of Rumor Believers', rumor_Believers / len(self.agents))
    self.report('Percentage of Truth Believers', truth_Believers / len(self.agents))
    # Anyone not included in this should be Ignorant (i.e., they have not heard the rumor nor the truth).

## Visualization

We prepared three functions for visualizations: a line plot, a stack plot, and an animation function.

In our presentation though, we only decided to retain the line plot and the animation.

In [4]:
# Line Plot
def rumor_lineplot(data, ax):
    x = data.index.get_level_values('t')
    y = [data[var] for var in ['I', 'RS', 'TS', 'RR', 'TR']]

    ax.plot(x,y[0], 'gray', label = "Ignorant")
    ax.plot(x,y[1], 'green', label = "Rumor Spreader")
    ax.plot(x,y[2], 'blue', label = "Truth Spreader")
    ax.plot(x,y[3], 'orange', label = "Removed RS")
    ax.plot(x,y[4], 'purple', label = "Removed TS")
    
    ax.legend()
    ax.set_xlim(0, max(1, len(x) - 1))
    ax.set_ylim(0, 1)
    ax.set_xlabel("Time Steps")
    ax.set_ylabel("Population")

# Stack Plot
def rumor_stackplot(data, ax):
    """ Stackplot of people's condition over time. """
    x = data.index.get_level_values('t')
    y = [data[var] for var in ['I', 'RS', 'TS', 'RR', 'TR']]

    sns.set()
    ax.stackplot(x, y, labels=['Ignorant', 'Rumor Spreader', 'Truth Spreader', 'Removed RS', 'Removed TS'],
                 colors = ['gray', 'green', 'blue', 'orange', 'purple'])

    ax.legend()
    ax.set_xlim(0, max(1, len(x) - 1))
    ax.set_ylim(0, 1)
    ax.set_xlabel("Time Steps")
    ax.set_ylabel("Percentage of population")

# Single-Run Animation
def animation_plot(model, axz):
    ax1, ax2 = axz # ax1, ax2, ax3 = axz
    attr_grid = model.city.attr_grid('condition')
    color_dict = {0:'#8F9491', 1:'#87FF65', 2:'#51CB20', 3:'#80FFEC', 4:'#266DD3', 5: '#054A29', 6: '#052F63', None:'#000000'}
    ap.gridplot(attr_grid, ax=ax1, color_dict=color_dict, convert=True)
    ax1.set_title(f"Simulation of the spread of a rumor\n"
                 f"Time-step: {model.t} I: ["
                 f"{len(model.agents.select(model.agents.condition == 0))}] "
                 "RS: ["
                 f"{len(model.agents.select(model.agents.condition == 1)) + len(model.agents.select(model.agents.condition == 2))}] "
                 "TS: ["
                 f"{len(model.agents.select(model.agents.condition == 3)) + len(model.agents.select(model.agents.condition == 4))}] "
                 "RR: ["
                 f"{len(model.agents.select(model.agents.condition == 5))}] "
                 "TR: ["
                 f"{len(model.agents.select(model.agents.condition == 6))}]"
                )
    
    rumor_lineplot(model.output.variables.RumorModel, ax2)

## Sample Run
Below is a sample run of the model using the default parameters.

Note that this is not the actual trial.

In [5]:
# Default parameters
p = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 0,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.86,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20, 6))
model = RumorModel(p)
animation = ap.animate(model, fig, axs, animation_plot)

The next two lines of code will display the visualizations in the notebook or save a gif file of the visualization to your device.

Only one of the two should be run.

In [8]:
IPython.display.HTML(animation.to_jshtml(fps=15))

In [ ]:
animation.save('Test0_Trial1.gif')

For ease of extracting the data from each trial, we use `model.output.variables.RumorModel` to see the percentage of each population belonging to each agent state for each time step. This is saved under the variable `df`.

In [9]:
df = model.output.variables.RumorModel
# Rumor Believers
df['RB'] = df['RS'] + df['RR']
# Truth Believers
df['TB'] = df['TS'] + df['TR']

print(df)

# Obtaining the percentages of the population at the end of the trial
print("Percentage of Each State at the End of the Simulation")
print(df.iloc[df.shape[0] - 1])

# Obtaining the peak value and time step for RB & TB 
print("Peak for Rumor Believers")
print(df.sort_values(by = 'RB', ascending = False)['RB'][:1])
print("Peak for Truth Believers")
print(df.sort_values(by = 'TB', ascending = False)['TB'][:1])

            I        RS        TS        RR        TR        RB        TB
t                                                                        
0    1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
1    0.999259  0.000370  0.000370  0.000000  0.000000  0.000370  0.000370
2    0.995556  0.001852  0.001852  0.000370  0.000370  0.002222  0.002222
3    0.990000  0.005185  0.003704  0.000370  0.000741  0.005556  0.004444
4    0.977778  0.011111  0.008148  0.001481  0.001481  0.012593  0.009630
..        ...       ...       ...       ...       ...       ...       ...
296  0.160000  0.238519  0.148519  0.272593  0.180370  0.511111  0.328889
297  0.155185  0.241852  0.147037  0.271481  0.184444  0.513333  0.331481
298  0.148889  0.246296  0.148889  0.273704  0.182222  0.520000  0.331111
299  0.162593  0.234815  0.141852  0.279630  0.181111  0.514444  0.322963
300  0.169630  0.233333  0.140000  0.287037  0.170000  0.520370  0.310000

[301 rows x 7 columns]
Percentage of 

## Tests

The following lines of code show the parameters for each conducted test.

The actual trials were done separately, and are not located within this notebook.

In [ ]:
# Test 1
# Decreased Probability of Accepting the Truth = 0.215
p1 = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 0,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.215,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p1)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test1_Trial 1.gif')

In [ ]:
# Test 2
# Decreased Probability of Accepting the Truth = 0.430
p2 = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 0,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.430,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p2)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test2_Trial1.gif')

In [ ]:
# Test 3
# Decreased Probability of Accepting the Truth = 0.645
p3 = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 0,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.645,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p3)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test3_Trial1.gif')

In [ ]:
# Test 4
# Increased Probability of Accepting the Truth = 1.0
p4 = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 0,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 1.00,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p4)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test4_Trial1.gif')

In [ ]:
# Test 5
# Increased Truth Spread Start = 1
p5 = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 1,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.86,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p5)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test5_Trial1.gif')

In [ ]:
# Test 6
# Increased Truth Spread Start = 10
p6 = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 10,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.86,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p6)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test6_Trial1.gif')

In [ ]:
# Test 7
# Increased Truth Spread Start = 25
p7 = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 25,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.86,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p7)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test7_Trial1.gif')

In [ ]:
# Test 8
# Increased Truth Spread Start = 50
p8 = {
    'Population Density' : 0.75,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 50,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.86,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p8)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test8_Trial1.gif')

In [ ]:
# Test 9
# Decreased Population Density = 0.5
p9 = {
    'Population Density' : 0.50,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 0,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.86,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p9)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test9_Trial1.gif')

In [ ]:
# Test 10
# Increased Population Density = 1.0

p10 = {
    'Population Density' : 1.00,
    'size' : 60,
    'steps' : 300,
    'Initial Rumor Spreaders' : 1,
    'Initial Truth Spreaders' : 1,
    'Rumor Spread Start' : 0,
    'Truth Spread Start' : 0,
    'Probability of Accepting the Rumor' : 0.86,
    'Probability of Accepting the Truth' : 0.86,
    'Probability of Switching Sides after Contact with a Truth Spreader' : 0.4,
    'Probability of Switching Sides after Contact with a Rumor Spreader' : 0.4,
    'Probability of Losing Interest for Rumor Spreaders' : 0.3,
    'Probability of Losing Interest for Truth Spreaders' : 0.3,
    'Probability of Forgetting the Rumor' : 0.25,
    'Probability of Forgetting the Truth' : 0.25     
}

fig, axs = plt.subplots(1, 2, figsize=(20,6))
model = RumorModel(p10)
animation = ap.animate(model, fig, axs, animation_plot)
animation.save('Test10_Trial1.gif')